In [ ]:
import numpy as np
import pandas as pd

from linear_regression import Model
from preprocessing import read_sequence_ids, read_original_predictions

# Data pre-processing

In [ ]:
sequences = read_sequence_ids("data/3U_sequences_final.txt")

# Linear Regression

In [ ]:
model_a_plus = Model.load("data/run_linear_3U_40A_dg_BEST.out.mat")
model_a_minus = Model.load("data/run_linear_3U_00Am1_dg_BEST.out.mat")

In [ ]:
prediction_a_minus = np.log2(model_a_minus.predict(sequences["sequence"]))

In [ ]:
prediction_a_plus = np.log2(model_a_plus.predict(sequences["sequence"]))

In [ ]:
prediction_df = pd.DataFrame({"id": sequences["id"],
                              "a_minus": prediction_a_minus.T[0],
                              "a_plus": prediction_a_plus.T[0]})
prediction_df.sort_values(by=["id"], inplace=True)

In [ ]:
compare_df, a_minus_clip, a_plus_clip = read_original_predictions("data/models_full_dg.txt")

In [ ]:
prediction_df["a_minus"].fillna(a_minus_clip, inplace=True)
prediction_df["a_minus"].clip(lower=a_minus_clip, inplace=True)

prediction_df["a_plus"].fillna(a_plus_clip, inplace=True)
prediction_df["a_plus"].clip(lower=a_plus_clip, inplace=True)

In [ ]:
np.allclose(prediction_df["a_minus"].to_numpy(), compare_df["a_minus"].to_numpy(), atol=1e-4, rtol=0, equal_nan=True)

In [ ]:
np.allclose(prediction_df["a_plus"].to_numpy(), compare_df["a_plus"].to_numpy(), atol=1e-4, rtol=0, equal_nan=True)